In [1]:
#from tabnet_scraper.tabnet_driver import TabnetDriver
from tabnet_scraper.schema_components import *
#driver = TabnetDriver()

In [3]:
from unidecode import unidecode
from typing import List
from selenium import webdriver
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager


from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.chrome.options import Options

from tabnet_scraper.schema_components import TabnetComponents

class TabnetDriver:

    def __init__(self, extractions: TabnetComponents):

        self.extractions = extractions
        options = Options()
        self.driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()),options=options)

    @staticmethod
    def select_options(item:str, component: list, driver):
        helpers = {
            'row': 'L',
            'column': 'C',
            'incremet': 'I'
        }
        
    
        item_html_element = Select(driver.find_element(by=By.ID, value=helpers.get(item)) )
        

        lower_components = [unidecode(comp).lower() for comp in component]
        items_to_click = [option for option in item_html_element.options if unidecode(option.text).lower() in lower_components]
    
        
        item_html_element.deselect_all
        for to_select in items_to_click:
            item_html_element.select_by_value(to_select.get_property('value'))
    
        return None 

    @staticmethod
    def select_custom_filters(extraction_filters:List[dict], driver):
        options = driver.find_element(by=By.CLASS_NAME, value='opcoes')
        filters = options.find_elements(by=By.CLASS_NAME, value='titulo_select')
    
        for custom_filter in filters:
            aux = unidecode(custom_filter.find_element(by=By.TAG_NAME, value='label').text)
            #if aux == 'Diagnostico Detalhado':
            custom_filter.find_element(by=By.TAG_NAME, value='img').click()
            select_custom_filter_options = Select(custom_filter.find_element(by=By.CLASS_NAME, value='fundo_select_tabnet'))
            select_custom_filter_options.deselect_all
            for filter_value in extraction_filters:
                select_custom_filter_options.select_by_visible_text(filter_value['values'])


        
    
    def extract(self, url:str):

        self.driver.get(url)
        
        self.current_page = self.driver.current_window_handle
        
        for index, component in enumerate(self.extractions.components):
            #row
            self.select_options(item='row', component=component.row,driver= self.driver)
            #col
            self.select_options(item='column', component=component.column,driver=self.driver)
            #mesure
            self.select_options(item='incremet', component=component.mesure,driver=self.driver)
            #custom filters
            self.select_custom_filters(component.custom_filters, self.driver)

            tabs = self.driver.window_handles
            for tab in tabs:
                if(tab!=self.current_page):
                    self.driver.switch_to.window(tab)
                    
            WebDriverWait(self.driver, 60).until(EC.presence_of_element_located((By.LINK_TEXT, 'SALVA COMO CSV')))    
            self.driver.find_element(by=By.LINK_TEXT,value='SALVA COMO CSV').click()

            yield {
                "id": component.id,
                "index": index,
                "row": component.row,
                "column": component.col,
                "measure": component.mesure,
                "filters": component.custom_filters,
                "status": True
             }
            
        self.driver.quit()
        
    

In [5]:


component = TabnetComponent(
        id='cancer_de_mama_painel_oncologia',
        row=['Município da residência'],
        column=['Sexo'],
        mesure=['Casos'],
        period=['2023'],
        custom_filters = [
            {'name': 'Diagnóstico Detalhado','values': ['C53 - Neoplasia maligna do colo do útero']},
            {'name': 'Sexo','values': ['Feminino']}
        ]
)
scraper = TabnetDriver(extractions=TabnetComponents(components = [component]))



In [6]:
driver = scraper.driver

In [9]:
driver.get('http://tabnet.datasus.gov.br/cgi/dhdat.exe?PAINEL_ONCO/PAINEL_ONCOLOGIABR.def')
options = driver.find_element(by=By.CLASS_NAME, value='opcoes')
filters = options.find_elements(by=By.CLASS_NAME, value='titulo_select')

In [12]:
unidecode(filters[0].find_element(by=By.TAG_NAME, value='label').text)

'Regiao - residencia'

In [64]:
driver = driver.driver
driver.get('http://tabnet.datasus.gov.br/cgi/dhdat.exe?PAINEL_ONCO/PAINEL_ONCOLOGIABR.def')

In [160]:

class TabnetComponent(BaseModel):
    id: str
    column: List[str] = Field(description='')
    row: List[str] = Field(description='')
    mesure: str = Field(description='')
    period: List[str] = Field(description='')
    custom_filters: List[dict] = Field(description='', options=True)


In [187]:
component = TabnetComponent(
    id='cancer_de_mama_painel_oncologia',
    row=['Município da residência'],
    column=['Sexo'],
    mesure='Casos',
    period=['2023'],
    custom_filters = [
        {'name': 'Diagnóstico Detalhado','values': ['C53 - Neoplasia maligna do colo do útero']},
        {'name': 'Sexo','values': ['Feminino']}
    ]
)


#tabnet_components = TabnetComponents(components = [component])

In [65]:
current_page_handle = driver.current_window_handle

In [191]:
current_page_handle

'3D8B39E6A7C83D656D80B10814A3388B'

In [ ]:
def select_options(item:str, component: list, driver):
    helpers = {
        'row': 'L',
        'column': 'C',
        'incremet': 'I'
    }

    item_html_element = Select(driver.find_element(by=By.ID, helpers.get(item)))
    lower_components = [unidecode(comp).lower() for comp in component]
    items_to_click = [option for option in item_html_element.options if unidecode(option.text).lower() in lower_components]

    
    item_html_element.deselect_all
    for to_select in items_to_click:
        item_html_element.select_by_value(to_select.get_property('value'))

    return None

Selecionando elementos da linha

In [103]:
rows_html_element = Select(driver.find_element(by=By.ID,value='L'))
lower_components = [unidecode(comp).lower() for comp in component.row]
rows_to_click = [option for option in rows_html_element.options if unidecode(option.text).lower() in lower_components]


rows_html_element.deselect_all
for to_select in rows_to_click:
    rows_html_element.select_by_value(to_select.get_property('value'))

Selecionando elementos da coluna

In [104]:
cols_html_element = Select(driver.find_element(by=By.ID,value='C'))
lower_components = [unidecode(comp).lower() for comp in component.column]
cols_to_click = [option for option in cols_html_element.options if unidecode(option.text).lower() in lower_components]


cols_html_element.deselect_all
for to_select in cols_to_click:
    cols_html_element.select_by_value(to_select.get_property('value'))

Selecionando elementos de incremento (medida)

In [110]:
mesures_html_element = Select(driver.find_element(by=By.ID,value='I'))
lower_components = [unidecode(component.mesure.lower()).lower()]
mesures_to_click = [option for option in mesures_html_element.options if unidecode(option.text).lower() in lower_components]



mesures_html_element.deselect_all
for to_select in mesures_to_click:
    mesures_html_element.select_by_value(to_select.get_property('value'))

In [114]:
if 'custom_filters' in component.model_fields:
    print('ok')

ok


In [121]:
from selenium.webdriver.common.action_chains import ActionChains

In [ ]:
def select_custom_filters(filters, driver):
    options = driver.find_element(by=By.CLASS_NAME, value='opcoes')
    filters = options.find_elements(by=By.CLASS_NAME, value='titulo_select')

    for custom_filter in filters:
        aux = unidecode(custom_filter.find_element(by=By.TAG_NAME, value='label').text)
        #if aux == 'Diagnostico Detalhado':
        custom_filter.find_element(by=By.TAG_NAME, value='img').click()
        select_custom_filter_options = Select(custom_filter.find_element(by=By.CLASS_NAME, value='fundo_select_tabnet'))
        select_custom_filter_options.deselect_all
        for filter_value in component.custom_filters[0]['values']:
            select_custom_filter_options.select_by_visible_text(filter_value)


In [188]:
x = driver.find_element(by=By.CLASS_NAME, value='opcoes')


#x = (
#    ActionChains(driver)
#    .move_to_element(driver.find_element(by=By.CLASS_NAME, value='selecoes'))
#    .move_to_element(driver.find_element(by=By.CLASS_NAME, value='corposelecoes'))
#    .move_to_element(driver.find_element(by=By.CLASS_NAME, value='opcoes'))
#    .perform()
#)

filters = x.find_elements(by=By.CLASS_NAME, value='titulo_select')


In [189]:
#custom filters
for custom_filter in filters:
    aux = unidecode(custom_filter.find_element(by=By.TAG_NAME, value='label').text)
    if aux == 'Diagnostico Detalhado':
        custom_filter.find_element(by=By.TAG_NAME, value='img').click()
        select_custom_filter_options = Select(custom_filter.find_element(by=By.CLASS_NAME, value='fundo_select_tabnet'))
        select_custom_filter_options.deselect_all
        for filter_value in component.custom_filters[0]['values']:
            select_custom_filter_options.select_by_visible_text(filter_value)

        

In [190]:
driver.find_element(by=By.CLASS_NAME,value='botoes').find_element(by=By.NAME, value='button1').click()

In [198]:
tabs = driver.window_handles
for tab in tabs:
    if(tab!=current_page_handle):
        driver.switch_to.window(tab)
#WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.LINK_TEXT, 'SALVA COMO CSV')))    

In [199]:
driver.find_element(by=By.LINK_TEXT,value='SALVA COMO CSV').click()

In [180]:
select_custom_filter_options.options[3].text

'C02 - Neoplasia maligna de outras partes e de partes não especificadas da língua'